In [1]:
include("JuliRay.jl")
Base.@irrational ° 0.0174532925199432957692369076848861271344 (big(pi)/big(180))

In [2]:
ε=10^(-9)

VERTEX=Int
EDGE=Array{VERTEX,1}
FACE=Array{EDGE,1}
CELL=Array{FACE,1}

function ℝ³⭢S³(p::Array{T,1}) where T<:Real
    if(length(p)≠3)
        error("No Point on ℝ³")
    end
    return [2p[1]/(1+p[1]^2+p[2]^2+p[3]^2),2p[2]/(1+p[1]^2+p[2]^2+p[3]^2),2p[3]/(1+p[1]^2+p[2]^2+p[3]^2),(-1+p[1]^2+p[2]^2+p[3]^2)/(1+p[1]^2+p[2]^2+p[3]^2)]
end

function ℝ³⭢S³(p::Array{T,1},θ::Real) where T<:Real
    if(length(p)≠3)
        error("No Point on ℝ³")
    elseif(θ<ε)
        return [p[1:3]...,0.0]
    else
        c=[0,0,0,cot(θ)]
        r=1/sin(θ)
        P=p*cot((π/2-θ)/2+π/4)
        return ℝ³⭢S³(P)*r+c
    end
end

function S³⭢ℝ³(q::Array{T,1}) where T<:Real
    if(length(q)≠4)
        error("No Point on S³")
    elseif(!(norm(q)≈1.0))
        print(norm(q))
        error("No Point on unit S³, $(norm(q))")
    end
    return [q[1]/(1-q[4]),q[2]/(1-q[4]),q[3]/(1-q[4])]
end

function S³⭢ℝ³(q::Array{T,1},θ::Real) where T<:Real
    if(length(q)≠4)
        error("No Point on S³")
    elseif(θ<ε)
        return q[1:3]
    else
        c=[0,0,0,cot(θ)]
        r=1/sin(θ)
        return S³⭢ℝ³((q-c)/r)*tan((π/2-θ)/2+π/4)
    end
end

function ℝ⁴⭢S³(p::Array{T,1},θ::Real) where T<:Real
    if(θ<ε)
        return [p[1:3]...,0.0]
    else
        R=1/sin(θ)
        O=[0,0,0,cot(θ)]
        return O+R*normalize(p-O)
    end
end

function Mirror(q::RealVector,p₁::RealVector,p₂::RealVector,p₃::RealVector)
    n=NormalVector(p₁,p₂,p₃)
    c=q-dot(n,q-p₁)*n
    return q-2*dot(n,q-p₁)*n
end

function NormalVector(p₁::RealVector,p₂::RealVector,p₃::RealVector,p₄::RealVector)
    A=hcat(p₁-p₄,p₂-p₄,p₃-p₄)
    return normalize([(-1)^i*det(A[deleteat!(collect(1:4),i),:]) for i ∈ 1:4])
end

function Mirror(q::RealVector,p₁::RealVector,p₂::RealVector,p₃::RealVector,p₄::RealVector)
    n=NormalVector(p₁,p₂,p₃,p₄)
    c=q-dot(n,q-p₁)*n
    return q-2*dot(n,q-p₁)*n
end

function vertices(face::FACE)
    f=copy(face)
    v=union(vcat(f...))
    w=[v[1]]
    for j ∈ 1:(length(v)-1)
        i=findfirst(e->w[end] ∈ e, f)
        push!(w,filter(v->v≠w[end],f[i])[1])
        deleteat!(f,i)
    end
    return w
end

function PickFace(cell::CELL, v::RealVector)
    if(norm(v)≈0)
        error("vector v must be non-zero")
    end
    faces=vertices.(cell)
    fpts=(i->POINTS³[i]).(faces)
    return cell[findmax([dot(v,+(pts...)) for pts ∈ fpts])[2]]
end

function NewCell(cell::CELL,v::RealVector)
    n=length(POINTS³)
    c=copy(cell)
    face=PickFace(cell,v)
    IND_face=vertices(face)
    IND_cell=Int[]
    for f ∈ c for e ∈ f for v ∈ e push!(IND_cell,v) end end end
    IND_cell=union(IND_cell)
    IND_cell2=[
        if(i ∈ IND_face)
            i
        else
            push!(POINTS³,Mirror(POINTS³[i],POINTS³[IND_face[1]],POINTS³[IND_face[2]],POINTS³[IND_face[3]]));
            if(θ<ε)
                push!(POINTS⁴,[POINTS³[end]...,0]);
            else
                push!(POINTS⁴,Mirror(POINTS⁴[i],O,POINTS⁴[IND_face[1]],POINTS⁴[IND_face[2]],POINTS⁴[IND_face[3]]));
            end
            n=n+1
        end
        for i ∈ IND_cell]
    return [[[IND_cell2[findfirst(w->w==v,IND_cell)] for v ∈ e] for e ∈ f] for f ∈ c]
end

function SphericalSphere(v,r::Real,θ::Real) where T<:RealVector
    V=S³⭢ℝ³(v,θ)
    return Sphere(V,r)
end

function SphericalCylinder(v₁,v₂,r::Real,θ::Real) where T<:RealVector
    w₁=ℝ⁴⭢S³((v₁+v₂)/2,θ)
    V₁=S³⭢ℝ³(v₁,θ)
    V₂=S³⭢ℝ³(v₂,θ)
    W₁=S³⭢ℝ³(w₁,θ)
    return Arc(V₁,W₁,V₂,r)
end

function SphericalPolygon(v::Array{T,1},θ::Real) where T<:RealVector
    n=length(v)
    u=ℝ⁴⭢S³(+(v...)/n,θ)
    v₁=v[1]
    v₂=v[2]
    v₃=v[3]
    V=(q->S³⭢ℝ³(q,θ)).(v)
    U=S³⭢ℝ³(u,θ)
    w=[ℝ⁴⭢S³((v[i]+v[mod(i,length(v))+1])/2,θ) for i ∈ 1:length(v)]
    W=(q->S³⭢ℝ³(q,θ)).(w)
    if(θ<ε)
        return Polygon(V)
    end
    
    O=Circumcenter(U,V[1],V[2],V[3])
    sphere=Sphere(O,norm(U-O))
    N=NormalVector(V[1],W[1],V[2]);
    direction=sign(dot(N,O-U))
    
    cylinder=csgIntersection([
            (V₁=V[i];
            V₂=V[mod(i,n)+1];
            V₃=V[mod(i+1,n)+1];
            C=Circumcenter(V₁,W[i],V₂);
            N=NormalVector(V₁,W[i],V₂);
            direction=sign(dot(N,U-W[i]));
            cylinder=Cylinder(C,C+2*direction*norm(U-O)*N,norm(U-O)))
            for i ∈ 1:n
            ])
    return csgClip(sphere,cylinder)
end

function Cells2Object(cells::Array{CELL,1},POINTS⁴;rᵥ=0.05,rₑ=0.025,color=RGB(1,1,1))
    cs=copy(cells)
    fs=DeleteDuplicates(vcat(cs...))
    es=DeleteDuplicates(vcat(fs...))
    vs=DeleteDuplicates(vcat(es...))
    V=rgbColor(csgUnion([SphericalSphere(POINTS⁴[v],rᵥ,θ) for v ∈ vs]),RGB(0.1,0.1,0.1))
    E=rgbColor(csgUnion([SphericalCylinder(POINTS⁴[e[1]],POINTS⁴[e[2]],rₑ,θ) for e ∈ es]),RGB(0.2,0.2,0.2))
    F=rgbftColor(csgUnion([SphericalPolygon([POINTS⁴[i] for i ∈ vertices(f)],θ) for f ∈ fs]),color,FT(0.1,0.3))
    return csgUnion(V,E,F)
end

Cells2Object (generic function with 1 method)

In [4]:
θ=0.2
    h=1/2
    r=√(1-h^2)

    𝒑₁=[1,1,1]
    𝒑₂=[1,1,-1]
    𝒑₃=[1,-1,1]
    𝒑₄=[1,-1,-1]
    𝒑₅=[-1,1,1]
    𝒑₆=[-1,1,-1]
    𝒑₇=[-1,-1,1]
    𝒑₈=[-1,-1,-1]
    POINTS=[𝒑₁,𝒑₂,𝒑₃,𝒑₄,𝒑₅,𝒑₆,𝒑₇,𝒑₈]
    POINTS=r*normalize.(POINTS)
    R=1/sin(θ)
    O=[0,0,0,cot(θ)]
    N=O+[0,0,0,R]
    H=O-[0,0,0,√(R^2-r^2)]
    POINTS³=copy(POINTS)
    POINTS⁴=[[𝒑...,0]+H for 𝒑 ∈ POINTS]

    v₁=1
    v₂=2
    v₃=3
    v₄=4
    v₅=5
    v₆=6
    v₇=7
    v₈=8
    e₁=[v₁,v₂]
    e₂=[v₁,v₃]
    e₃=[v₁,v₅]
    e₄=[v₂,v₄]
    e₅=[v₂,v₆]
    e₆=[v₃,v₄]
    e₇=[v₃,v₇]
    e₈=[v₄,v₈]
    e₉=[v₅,v₆]
    e₁₀=[v₅,v₇]
    e₁₁=[v₆,v₈]
    e₁₂=[v₇,v₈]
    f₁=[e₁,e₂,e₄,e₆]
    f₂=[e₁,e₃,e₅,e₉]
    f₃=[e₂,e₃,e₇,e₁₀]
    f₄=[e₄,e₅,e₈,e₁₁]
    f₅=[e₆,e₇,e₈,e₁₂]
    f₆=[e₉,e₁₀,e₁₁,e₁₂]
    c₁=[f₁,f₂,f₃,f₄,f₅,f₆]
    c₂=NewCell(c₁,[-1,0,0])
    c₃=NewCell(c₂,[-1,0,0])
    c₄=NewCell(c₁,[1,0,0])
    c₅=NewCell(c₁,[0,0,-1])
    c₆=NewCell(c₅,[0,-1,0])
    c₇=NewCell(c₆,[0,-1,0])
    c₈=NewCell(c₅,[0,1,0])
    
    cells=[c₁,c₂,c₃,c₄,c₅,c₆,c₇,c₈]
#     return cells, POINTS³, POINTS⁴
# end

8-element Array{Array{Array{Array{Int64,1},1},1},1}:
 [[[1, 2], [1, 3], [2, 4], [3, 4]], [[1, 2], [1, 5], [2, 6], [5, 6]], [[1, 3], [1, 5], [3, 7], [5, 7]], [[2, 4], [2, 6], [4, 8], [6, 8]], [[3, 4], [3, 7], [4, 8], [7, 8]], [[5, 6], [5, 7], [6, 8], [7, 8]]]                                                
 [[[9, 10], [9, 11], [10, 12], [11, 12]], [[9, 10], [9, 5], [10, 6], [5, 6]], [[9, 11], [9, 5], [11, 7], [5, 7]], [[10, 12], [10, 6], [12, 8], [6, 8]], [[11, 12], [11, 7], [12, 8], [7, 8]], [[5, 6], [5, 7], [6, 8], [7, 8]]]                              
 [[[9, 10], [9, 11], [10, 12], [11, 12]], [[9, 10], [9, 13], [10, 14], [13, 14]], [[9, 11], [9, 13], [11, 15], [13, 15]], [[10, 12], [10, 14], [12, 16], [14, 16]], [[11, 12], [11, 15], [12, 16], [15, 16]], [[13, 14], [13, 15], [14, 16], [15, 16]]]      
 [[[1, 2], [1, 3], [2, 4], [3, 4]], [[1, 2], [1, 17], [2, 18], [17, 18]], [[1, 3], [1, 17], [3, 19], [17, 19]], [[2, 4], [2, 18], [4, 20], [18, 20]], [[3, 4], [3, 19], [4, 20], [19, 20]

In [15]:
M=2
for i ∈ 0:M
    θ=i*π/2/M
    cells, POINTS³, POINTS⁴=Tesseract(θ)
    render(Cells2Object(cells,POINTS⁴),camera=LngLatCamera(lng=30°,lat=25°,pers=0.2,zoom=0.15,width=800,height=600),name="Tesseract",index=i)
end

BoundsError: BoundsError: attempt to access 8-element Array{Array{Float64,1},1} at index [9]

In [8]:
cells, POINTS³, POINTS⁴=Tesseract(0.2)

MethodError: MethodError: no method matching PickFace(::Array{Array{Array{Int64,1},1},1}, ::Array{Int64,1}, ::Array{Array{Float64,1},1})
Closest candidates are:
  PickFace(::Array{Array{Array{Int64,1},1},1}, ::Array{#s1,1} where #s1<:Real, ::Any, !Matched::Any) at In[6]:90
  PickFace(::Array{Array{Array{Int64,1},1},1}, ::Array{#s1,1} where #s1<:Real) at In[2]:90

In [5]:
render(Cells2Object([c₁,c₂,c₃,c₄,c₅,c₆,c₇,c₈],POINTS⁴),camera=LngLatCamera(lng=30°,lat=25°,pers=0.2,zoom=0.15,width=640,height=360),name="hgoe3")

povray: cannot open the user configuration file /home/horikawa/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Process(`povray hgoe3.pov`, ProcessExited(0))